In [1]:
!pip install spotipy --upgrade

In [2]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials as cc
import numpy as np
import pandas as pd
from IPython.display import display
import warnings
import time
import json
from pprint import pprint
from spotipy.oauth2 import SpotifyClientCredentials

In [8]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id = '85bc9d29761941f2a0dafdc9aa8a4903',client_secret = 'e1ee091ccf0b44b6a0dc243abb2f1414'))

In [4]:
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
print(playlists.shape)

(5000, 12)


In [5]:
songs = []
for i in range(len(playlists)):
    tracks = playlists.iloc[i].loc['tracks']
    for track in tracks:
        songs.append(track)

songs_df = pd.DataFrame(songs)
print(songs_df.shape)
songs_df.head()

(334487, 8)


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Lost Frequencies,spotify:track:2vCtiBvJJZfz773yTfAxPP,spotify:artist:7f5Zgnp2spUuuzKplmRkt7,What Is Love 2016 - Mike Mago Remix,spotify:album:4N1e8k2o2NN932Y8xgZV1p,209882,What Is Love 2016
1,1,John Legend,spotify:track:5WOLZP8KrXiupBjG1SSN5U,spotify:artist:5y2Xq6xcjJb2jVM54GHK3t,Love Me Now - Dave Audé Remix Radio Edit,spotify:album:2CFZOjwtV7B6XzULEwIdHN,219106,Love Me Now (Remixes)
2,2,July Child,spotify:track:0oQDQ9QiqsO63EEBAro8Le,spotify:artist:3Jghk94Gog2dhBAO7ZZaM8,Thinkin of U,spotify:album:2i0oT2HtRN4o3P45zQjel7,194000,Thinkin of U
3,3,Tiësto,spotify:track:7MUS0La2IQ85vJ59fQqtoN,spotify:artist:2o5jDhtHVPhrJdv3cEQ99Z,I Will Be Here - (Wolfgang Gartner Remix),spotify:album:2qY9UxNEbQR2O2ECc3Fd48,237066,Kaleidoscope Remixed
4,4,Joe Stone,spotify:track:3ciyZYofjiqmMUElM5qgGB,spotify:artist:4kwEd1P9j15ZqUVP5zK7Pv,The Party (This Is How We Do It),spotify:album:3SBDe6WBbEXNtjraPxXLOh,185807,The Party (This Is How We Do It)


In [6]:
songs_df = songs_df.drop_duplicates('track_uri')
print(songs_df.shape)

(108870, 8)


In [13]:
songs = pd.read_csv('recommender-systems/allsongs.csv')




# result_artist = sp.artist(songs_df['artist_uri'][0])
# result_song = sp.audio_features(songs_df['track_uri'][0])
# result_song = sp.track(songs_df['track_uri'][0])
# print(result_artist)
# print()
# print("--------------------------------------------")
# print()
# print(result_song)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
pop = []
sleep_min = 1
sleep_max = 2
start_time = time.time()
request_count = 0
for i in range(0,3344):
        if request_count % 13 == 0:
            time.sleep(np.random.uniform(sleep_min, sleep_max))
        else:
            tracks_to_get = songs_df.iloc[request_count:request_count+100]["track_uri"] #get the uri of these songs to get audio features
            pop.extend(sp.tracks(tracks_to_get)['popularity'])
        request_count+=100
songs["popularity"] = pop